In [2]:
%config IPCompleter.greedy = True

In [1]:
sc

In [3]:
lines=sc.textFile("input/file1")
lines

input/file1 MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2

In [4]:
lines.collect()

[u'Hello World Bye World']

In [5]:
words=lines.flatMap(lambda l:l.split(" "))

In [139]:
words.count()

4

In [104]:
words.collect()

[u'Hello', u'World', u'Bye', u'World']

# Problem 1

In [6]:
rdd1=sc.wholeTextFiles("input/*")

In [35]:
rdd1.take(10)

[(u'file:/home/vagrant/csds-material-master/input/file2',
  u'Hello Hadoop Goodbye Hadoop\n'),
 (u'file:/home/vagrant/csds-material-master/input/file1',
  u'Hello World Bye World\n')]

In [7]:
list1=rdd1.flatMapValues(lambda l:l.strip("\n").split(" ")).values().countByValue()

In [8]:
list1

defaultdict(<type 'int'>, {u'World': 2, u'Bye': 1, u'Hello': 2, u'Goodbye': 1, u'Hadoop': 2})

# Problem 2

In [9]:
rdd2=sc.textFile("hive/purchases.txt")

In [76]:
raw2_1=rdd2.map(lambda l:l.split(","))
raw2_2=raw2_1.map(lambda p:(p[0],p[1],p[2],float(p[3]),p[4]))


In [54]:
from pyspark.sql.types import *

In [71]:
fields=StructType([StructField("timestamp",StringType()),
                  StructField("location",StringType()),
                  StructField("category",StringType()),
                  StructField("price",FloatType()),
                  StructField("card",StringType())])

In [83]:
purchases=sqlContext.createDataFrame(raw2_2,fields)
purchases.persist()

DataFrame[timestamp: string, location: string, category: string, price: float, card: string]

In [89]:
purchases.registerAsTable('purchases')

1.Average price of products that were purchased via MasterCard

In [95]:
sqlContext.sql('select avg(price) from purchases where card="MasterCard"').show()

_c0              
275.0677317417774


2.Data recorded with highest total sale

In [100]:
sqlContext.sql('select TO_DATE(timestamp),sum(price) as sum_price from purchases group by TO_DATE(timestamp) order by sum_price desc limit 5').show()

_c0        sum_price         
2012-03-17 2384.480026245117 
2012-03-15 2144.109969139099 
2012-05-06 2093.20001411438  
2012-03-16 2083.8500175476074
2012-05-25 1990.0700035095215


So 2012-03-17 is of highest total sales.

3.Minimum value of a product under the Computers category

In [101]:
sqlContext.sql('select price from purchases where category="Computers" order by price limit 5').show()

price
0.38 
31.99
45.64
47.13
48.76


Minimum value of a product under the Computers category is 0.38

 4.Distinct categories of products

In [104]:
 sqlContext.sql('select count(1) from (select distinct category from purchases) t').show()

_c0
18 


5.Store location had the lowest total sales

In [107]:
 sqlContext.sql('select sum(price) as sum_price,location from purchases group by location order by sum_price limit 5').show()

sum_price          location
784.9599838256836  Plano   
866.7500038146973  Denver  
873.0800170898438  Spokane 
1021.6599979400635 Dallas  
1025.8199882507324 Fremont 


So Plano has lowest total sales with about 785

# Problem 3

In [114]:
raw2_2.take(5)

[(u'2012-07-20 09:59:00', u'Corpus Christi', u'CDs', 327.91, u'Cash'),
 (u'2012-03-11 17:29:00', u'Durham', u'Books', 115.09, u'Discover'),
 (u'2012-07-31 11:43:00', u'Rochester', u'Toys', 332.07, u'MasterCard'),
 (u'2012-06-18 14:47:00', u'Garland', u'Computers', 31.99, u'Visa'),
 (u'2012-03-27 11:40:00', u'Tulsa', u'CDs', 452.18, u'Discover')]

In [119]:
purchasesDF=raw2_2.toDF(['timestamp','location','category','price','card'])
purchasesDF.show(5)

timestamp           location       category  price  card      
2012-07-20 09:59:00 Corpus Christi CDs       327.91 Cash      
2012-03-11 17:29:00 Durham         Books     115.09 Discover  
2012-07-31 11:43:00 Rochester      Toys      332.07 MasterCard
2012-06-18 14:47:00 Garland        Computers 31.99  Visa      
2012-03-27 11:40:00 Tulsa          CDs       452.18 Discover  


In [123]:
from pyspark.sql.functions import *

1.Average price of products that were purchased via MasterCard

In [120]:
purchasesDF.filter(purchasesDF.card == "MasterCard").agg(avg(col('price'))).show()

AVG(price)        
275.06773195876286


2.Data recorded with highest total sale

In [128]:
sc.version

u'1.3.1'

In [126]:
#no to_date function in Spark 1.3.1 ,which is after 1.5.0. 
#Not sure how to solve this problem


NameError: name 'to_date' is not defined

3.Minimum value of a product under the Computers category

In [135]:
purchasesDF.filter(purchasesDF.category == "Computers")\
.select(purchasesDF.price).orderBy(purchasesDF.price).take(1)

[Row(price=0.38)]

4.Distinct categories of products

In [143]:
purchasesDF.agg(countDistinct(purchasesDF.category).alias("c")).collect()

[Row(c=18)]


5.Store location had the lowest total sales

In [149]:
purchasesDF.groupBy(purchasesDF.location).sum("price").orderBy("SUM(price)").take(5)

[Row(location=u'Plano', SUM(price)=784.96),
 Row(location=u'Denver', SUM(price)=866.75),
 Row(location=u'Spokane', SUM(price)=873.0799999999999),
 Row(location=u'Dallas', SUM(price)=1021.66),
 Row(location=u'Fremont', SUM(price)=1025.82)]

So Plano has lowest total sales with about 785